# NYPD_Use_of_Force__Subjects:

a. To get the required columns, use this module: 


1.   get_area_of_interest(df_spark, interested_columns)


b. Preprocessing pipeline: Pass your data through these functions. (if your columns fall in those categories)

1.   valid_date_check(date)
2.   valid_time_check(time)
3.   reverse_geo_code_boros(df_spark, Latitude, Longitude, Boro, lat_index, long_index)
4.   refine_age_group_race(df_spark, victim_age_group=None, suspect_age_group=None, suspect_race=None, victim_race=None)
5.   refine_sex_gender_impute(df_spark, suspect_age=None, suspect_gender=None, victim_age=None, victim_gender=None)
6.   refine_precinct_jur(df_spark, precinct=None, Jur_code=None)



In [ ]:
!pip install pyspark
!pip install openclean

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 52.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=ae6500e9ac17b2562d019154714eac6a57803139300c2bd9953322481efc3328
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
     |████████████████████████████████| 267 kB 30.8 MB/s 
     |████████████████████████████████| 109 kB 53.3 MB/s 
     |████████████████████████████████| 137 kB 57.4 MB/s 
     |████████████████████████████████| 69 kB 6.7 MB/s 
     |████████████████████████████████| 260 kB 50.4 MB/s 
     |████████████████████████████████| 180 kB 62.4 MB/s 
     |████████████████████████████████| 206 kB 55.0 MB/s 
     |████████████████████████████████| 525 kB 46.6 MB/s 
     |████████████████████████████████| 596 kB 50.0 MB/s 
     |████████████████████████████████| 149 kB 72.3 MB/s 
     |██████

In [ ]:
#importing packages required
from pyspark import SparkContext, SparkConf
import os
import requests
from six.moves import urllib
import sys 
import pandas as pd
import matplotlib 
import matplotlib as plt
import numpy as np
import scipy as sp
import IPython
from IPython import display
import sklearn
import random
import time
import warnings
import re
import matplotlib.pyplot as plt
%matplotlib inline
from openclean.pipeline import stream
from openclean.profiling.column import DefaultColumnProfiler
from openclean.data.source.socrata import Socrata
from openclean.pipeline import stream
from openclean.function.eval.datatype import IsDatetime
import datetime
import pandas as pd
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import StringType

In [ ]:
from geopy.geocoders import ArcGIS
geocoder=ArcGIS()
#example:
geocoder.reverse('40.61157006600007, -73.74736517199995')

Location(11-64 Redfern Ave, Far Rockaway, New York 11691, USA, (40.61161616586613, -73.74738361194636, 0.0))

In [ ]:
#Creating Spark Session
sc = SparkContext.getOrCreate();
spark = SparkSession(sc)

In [ ]:
#Downloading file from NYC Open Data
#https://drive.google.com/uc?export=download&id=1KoewEF59cC7WTOtLvRwxZjujSEtG4Vst

fn_src = 'https://drive.google.com/uc?export=download&id=1KoewEF59cC7WTOtLvRwxZjujSEtG4Vst'
fn_dst = '/content/NYPD_Use_of_Force_Incidents.csv'

from six.moves import urllib

if os.path.isfile(fn_dst):
    print('File has already been downloaded', fn_dst)
else:
    print('Fetching file. This may take a while...', fn_dst)
    urllib.request.urlretrieve(fn_src, fn_dst)
    print('File %s has been downloaded' % fn_dst)

File has already been downloaded /content/NYPD_Use_of_Force_Incidents.csv


In [ ]:
src = 'https://data.beta.nyc/dataset/0ff93d2d-90ba-457c-9f7e-39e47bf2ac5f/resource/7caac650-d082-4aea-9f9b-3681d568e8a5/download/nyc_zip_borough_neighborhoods_pop.csv'
dst = 'nyc_zip_borough_neighborhoods_pop.csv'

#https://data.cityofnewyork.us/resource/h9gi-nx95.csv

from six.moves import urllib

if os.path.isfile(dst):
    print('File %s has already been downloaded' % dst)
else:
    urllib.request.urlretrieve(src, dst)
    print('File %s has been downloaded' % dst)

File nyc_zip_borough_neighborhoods_pop.csv has already been downloaded


In [ ]:
#similarly, lets get them into pyspark rdd
def get_area_of_interest(df_spark, interested_columns):
  df_spark=df_spark.select(interested_columns)
  return df_spark

# 2. Module for date related columns

As the dataset is for the data from 2006 to 2020, we can see that there is data from unknown format of "1010-05-14" to the year 2020. We need to clean this. Over here, we remove the null values where the complaint date is <2006. 

In [ ]:
# fileName='1010-05-14 00:00:00'
# # matches=re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})", fileName)
# re.search(r'([0-9]{4}\-[0-9]{2}\-[0-9]{2})', fileName).group(0)

def valid_date_check(date):
  if date==None or date==" " or date=="":
      return False
  else:
    date_cpy=date
    date=date.split("-")
    try:
      month=int(date[0])
      day= int(date[1])
      year=int(date[2])
      if year>=2006 and year<=2020:
        try:
          refined_date=datetime.datetime(year, month, day)
          return True
        except:
          return False
      else:
        return False
    except:
      return False

# 3. Module for time related columns

Similarly, lets check for the time as well. Here we must have time between 
the standard 24 hours.

In [ ]:
#Deleting invalid time
def valid_time_check(time):
  if time==None or time==" " or time=="":
    return False
  else :
    cpy_time=time
    time=time.split(":")
    try:
      hour=int(time[0])
      mins=int(time[1])
      secs= int(time[2])
      # if hours is 24 then change it to 0 hours
      if hour == 24 and mins== 0 and secs == 0:
        hour=0
      try:
        newTime= datetime.time(hour,mins,secs)
        return True
      except :
        return False
    except:
      return False

#4. Module for Age Group and Race columns
The module works for only those columns whose column names are passed

In [ ]:
def refine_age_group_race(df_spark, victim_age_group=None, suspect_age_group=None, suspect_race=None, victim_race=None):
  #params: dataframe, col names for the respective age, gender cols
  if victim_age_group:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[victim_age_group])
  if suspect_age_group:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[suspect_age_group])
  if suspect_race:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[suspect_race])
  if victim_race:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[victim_race])
  return df_spark

# 5. Module for Gender, Race Columns for suspects and victims

The module works for only those columns whose column names are passed

In [ ]:
def refine_sex_gender_impute(df_spark, suspect_age=None, suspect_gender=None, victim_age=None, victim_gender=None):
  #params: dataframe, col names for the respective age, gender cols
  if suspect_age:
    if dict(df_spark.dtypes)[suspect_age]=='int':
      df_spark=df_spark.na.fill(0,subset=[suspect_age])
    else:
      df_spark=df_spark.na.fill("U",subset=[suspect_age])
  if victim_age:
    df_spark=df_spark.na.fill("U",subset=[victim_age])
  if suspect_gender:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[suspect_gender])
  if victim_gender:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[victim_gender])
  return df_spark

#NYPD Emergency_Response_Incidents 11085 Rows of data.

Confidence Level:	95% 
Confidence Interval: 5
Population:	
11000
      
Sample size needed:	
350-400


In [ ]:
df_spark=spark.read.option("header",True).csv(fn_dst,inferSchema=True)
print(df_spark.count())
df_spark=df_spark.sample(0.03290)
df_spark.count()

11385


380

# PROFILING TO CHECK FOR NULL VALUES IN ALL THE COLUMNS

In [ ]:
pandasDF = df_spark.toPandas()
ds=stream(pandasDF)
profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
TRI Incident Number,380,0,379,0.997368,8.564592
Subject Injury Level,380,0,5,0.013158,1.055880
Subject Injured,380,0,2,0.005263,0.874762
Age,380,11,49,0.132791,5.209970
Subject Race,380,0,6,0.015789,1.655999
Subject Gender,380,0,3,0.007895,0.683320
Force Against MOS,380,0,5,0.013158,0.714400
Subject Used Force,380,0,2,0.005263,0.509368


## a. Select the columns that are common with the original dataset:
**Incident_Number**.    ||      **Age**.      ||       **Subject_Race**.  ||      **Subject_Gender**

---

**We can consider the primary key along with this**
**"Incident Type"**


#RENAME COLUMN and Dropping Columns which are not similar to Main dataset

In [ ]:
from pyspark.sql.functions import upper, col
import pyspark.sql.functions as f
df_spark = df_spark.withColumnRenamed("TRI Incident Number","Incident_Number")
df_spark = df_spark.withColumnRenamed("Subject Race","Subject_Race")
df_spark = df_spark.withColumnRenamed("Subject Gender","Subject_Gender")
df_spark = df_spark.drop("Subject Injury Level")
df_spark = df_spark.drop("Subject Injured")
df_spark = df_spark.drop("Subject Used Force")
df_spark = df_spark.drop("Force Against MOS")
df_spark.show()

+---------------+---+------------+--------------+
|Incident_Number|Age|Subject_Race|Subject_Gender|
+---------------+---+------------+--------------+
|  2020044962170| 28|       BLACK|          MALE|
|  2020065962090| 28|       BLACK|          MALE|
|  2020079962143| 28|       BLACK|          MALE|
|  2021059962130| 28|       BLACK|          MALE|
|  2021061962033| 28|       BLACK|          MALE|
|  2021065962116| 28|       BLACK|          MALE|
|  2021066962153| 28|       BLACK|          MALE|
|  2021096962016| 28|       BLACK|          MALE|
|  2021020962089| 30|       BLACK|          MALE|
|  2021066962056| 30|       BLACK|          MALE|
|  2020086962197| 24|       BLACK|          MALE|
|  2021103962038| 24|       BLACK|          MALE|
|  2020024962048| 26|       BLACK|          MALE|
|  2020026962078| 26|       BLACK|          MALE|
|  2020029962052| 26|       BLACK|          MALE|
|  2020059962288| 26|       BLACK|          MALE|
|  2020063962233| 26|       BLACK|          MALE|


#b. Lets pass the dataset through the preprocessing pipeline

In [ ]:
df_temp=df_spark.rdd

In [ ]:
pandasDF = df_spark.toPandas()
ds=stream(pandasDF)

#Creating profile of our dataset
profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
Incident_Number,380,0,379,0.997368,8.564592
Age,380,11,49,0.132791,5.209970
Subject_Race,380,0,6,0.015789,1.655999
Subject_Gender,380,0,3,0.007895,0.683320


In [ ]:
df_spark=refine_age_group_race(df_spark, None, None, 'Subject_Race', None)

In [ ]:
df_spark=refine_sex_gender_impute(df_spark, 'Age', 'Subject_Gender', None, None)

In [ ]:
pandasDF = df_spark.toPandas()
ds=stream(pandasDF)

#Creating profile of our dataset
profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
Incident_Number,380,0,379,0.997368,8.564592
Age,380,0,50,0.131579,5.248240
Subject_Race,380,0,6,0.015789,1.655999
Subject_Gender,380,0,3,0.007895,0.683320
